In [1]:
from pprint import pprint
from random import random

import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

from parsing import *
from preprocessing import *
from filtrage import *
from common import *
from vectorization import *
from learning import *
from postprocessing import *

## Chargement des données + preprocessing

In [2]:
data_train, data_test = parse_movie_reviews()
classes_train = (["neg"] * 1000) + (["pos"] * 1000)
data_train, classes_train, data_test = apply_preprocessing()

In [3]:
count_vectorizer = CountVectorizer()
data_train_count = count_vectorizer.fit_transform(data_train)

tf_transformer = TfidfTransformer(use_idf=False).fit(data_train_count)
data_train_tf = tf_transformer.fit_transform(data_train_count)

tfidf_transformer = TfidfTransformer()
data_train_tfidf = tfidf_transformer.fit_transform(data_train_count)

# Transform test
data_test_count = count_vectorizer.transform(data_test)
data_test_tfidf = tfidf_transformer.transform(data_test_count)

# Apprentissage
classifieur = learn(data_train_tfidf, classes_train, MultinomialNB)

In [4]:
ppln_count_mnnaivebayes = Pipeline([
    ("vect", count_vectorizer),
    ("clf", MultinomialNB())
])

ppln_tf_mnnaivebayes = Pipeline([
    ("vect", count_vectorizer),
    ("tfidf", tf_transformer),
    ("clf", MultinomialNB())
])

ppln_tfidf_mnnaivebayes = Pipeline([
    ("vect", count_vectorizer),
    ("tfidf", tfidf_transformer),
    ("clf", MultinomialNB())
])

## Prédictions

In [10]:
# Prediction
predictions = classifieur.predict(data_test_count)
postprocessing_movie_reviews(predictions)

In [9]:
import numpy as np


OUTPUT_CM = "_OUTPUT_CM.txt"
OUTPUT_MR = "_OUTPUT_MR.txt"


def postprocessing_chirac_mitterrand(predictions):
    global OUTPUT_CM

    content = "\n".join(predictions)
    with open(OUTPUT_CM, "w") as file:
        file.write(content)


def postprocessing_movie_reviews(predictions):
    global OUTPUT_MR

    new_predictions = []
    for c in predictions:
        if c == "pos":
            new_predictions.append("M")
        else:
            new_predictions.append("C")
    predictions = new_predictions[:-1]

    np.savetxt(OUTPUT_MR, predictions, fmt="%c")

## On output la prédiction